<a href="https://colab.research.google.com/github/upascal/Historical-Computational-Social-Science/blob/main/batch_OCR_to_TXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

****Batch OCR PDF to TXT with Tesseract****

Code based on 
https://colab.research.google.com/drive/1UqCSFTFF4Pi3RxS-iGL7-H_sGIgccusO

changes:
Removed natas.
Added batch processing

# Install dependencies

In [5]:
%%shell
pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html -U
sudo apt-get install tesseract-ocr tesseract-ocr-por
sudo apt-get install -y libsm6 libxext6 libxrender-dev
sudo apt-get install -y poppler-utils

pip install pytesseract
pip install pdf2image

Looking in links: https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html
     |███████████████████████████████▏| 834.1MB 1.0MB/s eta 0:00:22tcmalloc: large alloc 1147494400 bytes == 0x39710000 @  0x7f2f9d947615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x5f7832 0x4e5493 0x5515c1 0x5a9dac 0x50a433 0x50cc96 0x5095c8 0x50a2fd 0x50beb4 0x507be4
     |████████████████████████████████| 857.1MB 20kB/s 
     |████████████████████████████████| 17.4MB 240kB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following 

In [ ]:
#NOTE: Natas doesn't work in colab because it requires an older version of pytorch 

#%%shell
#pip3 install natas
#python3 -m natas.download
#python3 -m spacy download en_core_web_md

#Imports

In [4]:

#Libraries for PDF image conversion 
from pdf2image import convert_from_bytes, convert_from_path
import pytesseract
import numpy as np
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow


#File Mangement
import re
import os
from google.colab import drive
from google.colab import files

ModuleNotFoundError: ignored

# Mount PDF Folder from Drive

In [3]:
#Mount Google Drive
drive.mount('/content/drive') #(click, copy, press enter)

NameError: ignored

In [ ]:
#Inspect Directory
!ls

drive  sample_data


In [ ]:
#NOTE: Use the site menu to find file path

#Change Directory to data folder
%cd /content/drive/My Drive/Dissertation/Data/Annual Reports/SEC

/content/drive/My Drive/Dissertation/Data/Annual Reports/SEC


In [ ]:
#Make new directory for text ouptut 
!mkdir text_output

mkdir: cannot create directory ‘text_output’: File exists


In [ ]:
#Set directory variables
directory = '/content/drive/My Drive/Dissertation/Data/Annual Reports/SEC'
new_directory = directory+"/text_output/"

# Define OCR Functions

In [ ]:
def binarization(image):
    """

    Function that applies binarization to the image.

    Parameters
    ----------
    image : PIL.Image.Image
        Image to be binarized.

    Returns
    -------
    PIL.Image.Image
        Binarized image.
    """ 
    image = image.convert('RGB')
    npimage = np.asarray(image).astype(np.uint8)  
    npimage[:, :, 0] = 0
    npimage[:, :, 2] = 0
    im = cv2.cvtColor(npimage, cv2.COLOR_RGB2GRAY) 
    thresh = cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,2)
    binimage = Image.fromarray(thresh) 
    return binimage


def get_pages_as_images(file):
    """
    Converts PDF pages into an image and performs binarization.
  
    Parameters
    ----------
    file : bytes
        Document in `bytes` containing the `PDF`.

    Returns
    -------
    PIL.Image.Image
        Document transformed into image and binariezed
    """ 
    images = convert_from_bytes(file, 250, grayscale=True)
    images_return = []
    num_page = 1
    for image in images:
        left = image.size[0]*0.05
        right = image.size[0]*0.95
        top = image.size[1]*0.05
        bottom = image.size[1]*0.95
        image = image.crop((left, top, right,bottom))
        images_return.append(binarization(image))
        
    return images_return

def get_ocr_documents(images):
    """

    Cycles through PDF images with pytesseract and converts images to string.
    
    Parameters
    ----------
    images : PIL.Image.Image
        Images resulting from PDF conversion.

    Returns
    -------
    String
        A text containing all pages from the PDF.
    """ 

    pages_text = []
    for image in images:
      #https://stackoverflow.com/questions/44619077/pytesseract-ocr-multiple-config-options
      pages_text.append(pytesseract.image_to_string(image, config='--psm 4' ,lang='eng'))
    return ''.join(pages_text)

def is_number(s):
    """

    Checks if the variable is a number
    
    Parameters
    ----------
    image : int
        The first parameter.

    Returns
    -------
    bool
        True if the type change occurs. False otherwise.
    """ 
    try:
        float(s)
        return True
    except ValueError:
        return False

# Run OCR on documents


In [ ]:
for filename in os.listdir(directory):
    
    if filename.endswith(".pdf"):
        print("Starting OCR on {}".format(filename))
        #Load PDF
        file_path = os.path.join(directory, filename)
        pdf_file = open(file_path, "rb").read()

        #OCR PDF
        images = get_pages_as_images(pdf_file) 
        pages_text = get_ocr_documents(images)
        print("Finished OCR")

        #Save to new Directory as TXT
        base_name = os.path.splitext(os.path.basename(filename))[0] #removes "PDF"
        output_file_name = "{}_OCR.txt".format(base_name) #creates output file name
        output_file_path = new_directory+output_file_name
        with open(output_file_path, 'w') as f:
          f.write(pages_text)
        print("Saved text as {}".format(output_file_name))

    else:
        print("{} is not a PDF".format(filename))
        continue





Starting OCR on U.S._Securities_and_Exchange_Commission_2006_Securities_and_Exchange.pdf
Finished OCR
Saved text as U.S._Securities_and_Exchange_Commission_2006_Securities_and_Exchange_OCR.txt
Starting OCR on U.S._Securities_and_Exchange_Commission_1959_25th_Annual_Report_of_Securities_and_Exchange_Commission.pdf
Finished OCR
Saved text as U.S._Securities_and_Exchange_Commission_1959_25th_Annual_Report_of_Securities_and_Exchange_Commission_OCR.txt
Starting OCR on U.S._Securities_and_Exchange_Commission_1964_30th_Annual_Report_1964.pdf
Finished OCR
Saved text as U.S._Securities_and_Exchange_Commission_1964_30th_Annual_Report_1964_OCR.txt
Starting OCR on U.S._Securities_and_Exchange_Commission_1965_31st_Annual_Report_1965.pdf
Finished OCR
Saved text as U.S._Securities_and_Exchange_Commission_1965_31st_Annual_Report_1965_OCR.txt
Starting OCR on U.S._Securities_and_Exchange_Commission_1966_32nd_Annual_Report_1966.pdf
Finished OCR
Saved text as U.S._Securities_and_Exchange_Commission_1966_3

Go to text_output Drive folder to view 

In [2]:
%cd directory+"/text_output/"

[Errno 2] No such file or directory: 'directory+/text_output/'
/content
